## Using ResNet50

In [1]:
import glob
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras import layers
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.python.keras import models
import tensorflow as tf

### Print the device configuration

In [2]:
tf.keras.backend.clear_session()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()[0], end='')

name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3214499273677409488


### Get the images of train dataset

In [3]:
def read_many(path):
    """
    Read all imagens in directory.
    
    Parameters
    ----------
    path: str
        Dataset path of a class (COVID or NON-COVID).
    
    Returns
    -------
    out : [ndarray]
        List of images.
    """
    # Get the path of all images
    list_imgs = list(glob.glob(path))
    out = []
    
    # Load all images of the given paths
    for i in range(len(list_imgs)):
        # Read the image in shape of (244, 244, 3)
        try:
            img = image.load_img(list_imgs[i], target_size=(224, 224, 3))
            x = image.img_to_array(img)
            out.append(x)
        # Print error
        except ValueError:
            print('Error reading the following image:', list_imgs[i])
    
    # Return the loaded images
    return out


def load_dir(paths):
    """
    Read images of COVID and NON-COVID cases.
    
    Parameters
    ----------
    paths: [str]
        Original and augmented dataset paths.
    
    Returns
    -------
    X : [ndarray]
        List of images.
    Y : [str]
        Labels of the images (i.e., 0 - NON-COVID; 1 - COVID).
    """
    # Arrays of images of COVID and NON-COVID cases
    covid = []
    non_covid = []
    
    # Read images
    for path in paths:
        # Read images of covid cases
        covid.extend(read_many('{}/COVID/*'.format(path)))
        # Read images of non-covid cases
        non_covid.extend(read_many('{}/NON_COVID/*'.format(path)))
    
    # Set COVID classes
    y_covid = np.asarray([1] * len(covid))
    y_non_covid = np.asarray([0] * len(non_covid))
    
    # Merge the read images
    X = np.concatenate([np.array(covid), np.array(non_covid)]) / 255
    Y = np.concatenate([np.array(y_covid), np.array(y_non_covid)])
    
    assert len(X) == len(Y), 'The number of images and the number of classes are different!'
    print('Images read:', len(X))
    
    # Return the read images and their labels
    return (X, Y)

In [4]:
TrainX, TrainY = load_dir(['Dataset/Train', 'Augmented/Train'])

Images read: 168


### Convolutional Neural Network

In [102]:
def CNN():
    """
    Return a Convolutional Neural Network (CNN) architecture.

    Returns
    -------
    model : Model
        The CNN model architeture.
    """
    # Create a new ResNet50
    conv_base = ResNet50(weights='imagenet',
                         # include_top = False,
                         # input_shape = (224,224, 3)
                        )
    # conv_base.summary()
    # Set the ResNet50 as trainable
    conv_base.trainable = True
    
    # model = conv_base
    # Instance of a sequential neural network
    model = models.Sequential()
    # Add the resnet50 to the sequential CNN
    model.add(conv_base)
    # Add a flaterns layer
    model.add(layers.Flatten())
    # Add a droput layer to avoid overfiting
    model.add(layers.Dropout(0.2))
    # Add relu layer for future classification
    model.add(layers.Dense(256, activation = 'relu'))
    # Add a final sigmoid layer for binary classification
    model.add(layers.Dense(1, activation = 'sigmoid'))
    # model.summary()
    
    # Compile the CNN model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy']      # model configuration
                 )
    
    # Return the built CNN model
    return model

In [103]:
model = CNN()

### Train the CNN

In [104]:
model.fit(x=TrainX, y=TrainY, epochs=10)

Epoch 1/10
6/6 [==============================] - 35s 5s/step - loss: 0.6761 - accuracy: 0.6905
Epoch 2/10
6/6 [==============================] - 31s 5s/step - loss: 0.6294 - accuracy: 0.7857
Epoch 3/10
6/6 [==============================] - 32s 5s/step - loss: 0.5873 - accuracy: 0.7560
Epoch 4/10
6/6 [==============================] - 31s 5s/step - loss: 0.5639 - accuracy: 0.7560
Epoch 5/10
6/6 [==============================] - 31s 5s/step - loss: 0.5292 - accuracy: 0.8214
Epoch 6/10
6/6 [==============================] - 31s 5s/step - loss: 0.4994 - accuracy: 0.8214
Epoch 7/10
6/6 [==============================] - 31s 5s/step - loss: 0.4747 - accuracy: 0.8214
Epoch 8/10
6/6 [==============================] - 34s 6s/step - loss: 0.4519 - accuracy: 0.8155
Epoch 9/10
6/6 [==============================] - 30s 5s/step - loss: 0.4208 - accuracy: 0.8512
Epoch 10/10
6/6 [==============================] - 32s 5s/step - loss: 0.4036 - accuracy: 0.8333
